In [33]:
import requests
import pandas as pd

# URLs de la API
DATA_URL = "https://api.ourworldindata.org/v1/indicators/1016584.data.json"
METADATA_URL = "https://api.ourworldindata.org/v1/indicators/1016584.metadata.json"
# Función para cargar los datos desde la API
def cargar_datos():
    datos = requests.get(DATA_URL).json()
    metadata = requests.get(METADATA_URL).json()
    return datos, metadata

def construir_dataframe(datos, metadata):
    # Extraer arrays principales
    valores = datos["values"]
    entidades = datos["entities"]
    anios = datos["years"]

    # Crear diccionario ID -> Nombre de país
    id_to_country = {
        ent["id"]: ent["name"]
        for ent in metadata["dimensions"]["entities"]["values"]
    }

    # Lista completa de años del metadata
    lista_anios = [y["id"] for y in metadata["dimensions"]["years"]["values"]]

    # Lista completa de países (nombres)
    lista_paises = [ent["name"] for ent in metadata["dimensions"]["entities"]["values"]]

    # Crear todos los pares posibles País × Año
    df_base = pd.DataFrame(
        [(pais, anio) for pais in lista_paises for anio in lista_anios],
        columns=["País", "Año"]
    )

    # Crear DataFrame con datos reales desde el JSON
    registros = []
    for i in range(len(valores)):
        pais_id = entidades[i]
        anio = anios[i]
        valor = valores[i]
        pais = id_to_country.get(pais_id)
        if pais:
            registros.append((pais, anio, valor))

    df_valores = pd.DataFrame(registros, columns=["País", "Año", "Valor"])

    # Merge con la tabla base (left join para conservar todos los países y años)
    df_final = pd.merge(df_base, df_valores, on=["País", "Año"], how="left")

    return df_final

# Uso
datos, metadata = cargar_datos()
df_pesticidas = construir_dataframe(datos, metadata)

# Mostrar verificación
print(df_pesticidas.head(10))
print(df_pesticidas[(df_pesticidas["País"] == "Belgium") & (df_pesticidas["Año"] == 1998)])

           País   Año     Valor
0  Africa (FAO)  1990  73504.69
1  Africa (FAO)  1991  69490.83
2  Africa (FAO)  1992  59769.13
3  Africa (FAO)  1993  53768.65
4  Africa (FAO)  1994  58312.42
5  Africa (FAO)  1995  64426.54
6  Africa (FAO)  1996  66678.48
7  Africa (FAO)  1997  71455.93
8  Africa (FAO)  1998  73631.57
9  Africa (FAO)  1999  75315.72
        País   Año  Valor
668  Belgium  1998    NaN


In [34]:
import pandas as pd

def limpiar_y_guardar_csv(df, nombre_archivo):
    # Eliminar filas con valores nulos
    df_limpio = df.dropna()

    # Guardar en un archivo CSV
    df_limpio.to_csv(nombre_archivo, index=False, encoding="utf-8")

    print(f"Datos guardados en '{nombre_archivo}' sin valores nulos.")
    
    return df_limpio  

In [35]:
limpiar_y_guardar_csv(df_pesticidas,"Datos_uso_pepticidas_p.csv")

Datos guardados en 'Datos_uso_pepticidas_p.csv' sin valores nulos.


,País,Año,Valor
0,Africa (FAO),1990,73504.69
1,Africa (FAO),1991,69490.83
2,Africa (FAO),1992,59769.13
3,Africa (FAO),1993,53768.65
4,Africa (FAO),1994,58312.42
...,...,...,...
8245,Zimbabwe,2018,2185.07
8246,Zimbabwe,2019,2185.07
8247,Zimbabwe,2020,2185.07
8248,Zimbabwe,2021,2185.07
